# VBA Benchmark Model

In [1]:
import numpy
import xlrd
from openpyxl import load_workbook
from vbaFunc import jumpoff, xTFR, clear_Input, readCNAPC, readSYI, readSATP

# Read in Excel Workbook (both Readable and Writable)
wt_loc = '../VBA_Benchmark/vba.xlsx'
rd_loc = '../VBA_Benchmark/original.xlsm'
wb_rd = xlrd.open_workbook(rd_loc)
wb_wt = load_workbook(wt_loc)

## xTRF

#### Generating Fertility Data

In [2]:
# Read in Sheets
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
wb_wt_fertility = wb_wt["Fertility"]

# Define variables
numareas = int(sheet_label.cell_value(2256, 0))
numages = 18
sextypes = 2
set_year_female = 7
set_year_male = 5

# Create empty array for storing jumpoffERP and xTFR results
jumpoffERP = numpy.zeros((numareas, sextypes, numages))
jumpoffERP[:] = numpy.nan
result_xTFR = numpy.zeros((numareas))
result_xTFR[:] = numpy.nan

# Generate jumpoffERP for further calculation
jumpoffERP = jumpoff(numareas, numages, jumpoffERP, sheet_agesex, set_year_female, set_year_male)

# Generate xTFR vector for storing
result_xTFR = xTFR(jumpoffERP, result_xTFR, numareas)

# Store into Sheet 'Fertility' for further usage
row = 7
column = 4
for i in range(len(result_xTFR)):
    wb_wt_fertility.cell(row, column).value = result_xTFR[i]
    row += 1
wb_wt.save(wt_loc)

## PrepareData

#### Clear Input Data

In [3]:
# Before running the PrepareData Function, clear the input data if any
wb_wt_SmallAreaInputs = wb_wt["SmallAreaInputs"]
wb_wt_Accounts = wb_wt["Accounts"]

# Clear the data and save the writable workbook
clear_Input(wb_wt_SmallAreaInputs)
clear_Input(wb_wt_Accounts)
wb_wt.save(wt_loc)

#### Load in Required Data for Generating Input Data

In [4]:
# Read in readable required data for generating input data for further usage
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_SmallAreaTotals = wb_rd.sheet_by_name('SmallAreaTotals')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
sheet_mortality = wb_rd.sheet_by_name('Fertility')
sheet_migration = wb_rd.sheet_by_name('Migration')

# Temeorary read of new generated fertility
temp_wb_rd = xlrd.open_workbook(wt_loc)
sheet_fertility = temp_wb_rd.sheet_by_name('Fertility')

#### Start Creating Input Data

In [5]:
# Define variables
numareas = int(sheet_label.cell_value(2256, 0)) # Number of small areas
numages = 18 # Number of age groups in ERPs
lastage = 17 # Number of age groups in projections output
final = int(sheet_label.cell_value(2271, 0)) # Final projection year number

# Read in AreaCode, AreaName, AgeGroup, Period-Cohort Labels
Areacode, Areaname, agelabel, pclabel = readCNAPC(numareas, numages, lastage, sheet_label)

# Read in Sex, Year, Interval Labels
sexlabel, yearlabel, intervallabel = readSYI(final, sheet_label)

# Read in Estimated & Projected Small Area Total Population (Aggregated)
TotPop = readSATP(2, sheet_SmallAreaTotals, numareas)

In [6]:
TotPop

array([[ 58408.        ,  19649.        ,  71611.        ,
         35853.        ,  36438.        , 172780.        ,
        160484.        ,  47018.        ,  56430.        ,
         47252.        ,  58200.        ,  51135.        ,
         87090.        ,  25271.        ,  20978.        ,
         70717.        ,  88630.        ,  74149.        ,
         72273.        ,  30847.        ,  77094.        ,
         91281.        , 131242.        ,  31570.        ,
         48950.        ,  79022.        ,  54319.        ,
         61752.        ,  12780.        ,  42529.        ,
         37943.        ,  38872.        ,  26518.        ,
         81730.        , 122953.        ,  76093.        ,
        168059.        ,  80130.        ,  71566.        ,
         92261.        ,  49136.        ,  14680.        ,
         94607.        , 100384.        ,  48141.        ,
        146731.        ,  26875.        ,  25118.        ,
         32462.        , 137027.        ,  91458.       